In [1]:
!pip show pynrrd nibabel

Name: pynrrd
Version: 1.0.0
Summary: Pure python module for reading and writing NRRD files.
Home-page: https://github.com/mhe/pynrrd
Author: Maarten Everts
Author-email: me@nn8.nl
License: MIT License
Location: C:\Users\jmunuera\AppData\Roaming\Python\Python311\site-packages
Requires: nptyping, numpy, typing-extensions
Required-by: 
---
Name: nibabel
Version: 5.2.1
Summary: Access a multitude of neuroimaging data formats
Home-page: 
Author: 
Author-email: NiBabel developers <neuroimaging@python.org>
License: MIT License
Location: C:\Users\jmunuera\AppData\Roaming\Python\Python311\site-packages
Requires: numpy, packaging
Required-by: dicom2nifti


In [ ]:
#!pip install pynrrd nibabel

In [2]:
import os
import glob
import nrrd #pip install pynrrd, if pynrrd is not already installed
import nibabel as nib #pip install nibabel, if nibabel is not already installed
import numpy as np

In [7]:
# Specify the directory containing .nrrd files
input_dir = r'D:\G-AI neuroimaging\test files\nrrd test'
# Check if the directory exists and list its contents
if os.path.exists(input_dir):
    print("Directory contents:", os.listdir(input_dir))
else:
    print("Directory does not exist. Check the path.")

# Get a list of .nrrd files in the directory
nrrd_files = glob.glob(os.path.join(input_dir, '*.nrrd'))

Directory contents: ['1075451.nrrd', '1174563.nrrd', '180818.nrrd', '556631.nrrd', '873308.nrrd']


In [9]:
# Prepare to log files with non-positive zooms or issues
skipped_files = []

# Assuming nrrd_files is a list of NRRD file paths
if nrrd_files:
    for nrrd_file in nrrd_files:
        try:
            # Read the .nrrd file
            data, header = nrrd.read(nrrd_file)

            # Check for label value 11 in the data
            if np.any(data == 11):
                # Create a mask for label value 11 ('Lobar')
                mask = data == 11
                # Apply the mask to isolate the region of interest and convert it to 1
                data = np.where(mask, 1, 0)  # Regions of interest set to 1, others to 0
            else:
                print(f"No 'Lobar' segment with label 11 found in {nrrd_file}")
                skipped_files.append(nrrd_file)
                continue  # Skip this file if the label is not found

            # Extract zoom values and check if they are positive
            zooms = [
                header['space directions'][0, 0],
                header['space directions'][1, 1],
                header['space directions'][2, 2]
            ]

            if any(z <= 0 for z in zooms):
                # Log this file for further review
                skipped_files.append(nrrd_file)
                continue  # Skip the current file

            # Create a NIfTI1Image object with the binary data
            nifti_img = nib.Nifti1Image(data, affine=None)

            # Update the NIfTI header with necessary information
            nifti_img.header.set_data_dtype(data.dtype)
            nifti_img.header.set_zooms(zooms)

            # Generate the output .nii file path by replacing the extension
            nii_file = nrrd_file.replace('.nrrd', '.nii.gz')

            # Save the NIfTI1Image object as .nii file
            nib.save(nifti_img, nii_file)
        except Exception as e:
            print(f"Error processing {nrrd_file}: {str(e)}")
else:
    print("No NRRD files found. Please check the directory and file extensions.")

# Report the skipped files
if skipped_files:
    print("Skipped files with issues:")
    for file in skipped_files:
        print(file)


In [5]:
import nrrd

# Function to print header information
def print_header_info(file_path):
    _, header = nrrd.read(file_path)
    for key, value in header.items():
        print(f"{key}: {value}")

# Paths to the files
problem_file = r'D:\G-AI neuroimaging\segmentations\nrrd\180818.nrrd'
normal_file = r'D:\G-AI neuroimaging\test files\seg\Segmentation_2.nrrd'

print("Problem File Header:")
print_header_info(problem_file)

print("\nNormal File Header:")
print_header_info(normal_file)


Problem File Header:
type: unsigned char
dimension: 3
space: left-posterior-superior
sizes: [512 638 197]
space directions: [[ 0.44672087 -0.00841547  0.03111792]
 [ 0.01685721  0.42950975 -0.12584205]
 [-0.04907886  0.2262862   0.76575991]]
kinds: ['domain', 'domain', 'domain']
encoding: gzip
space origin: [ -132.5622528  -312.1877757 -1147.662631 ]
Segment0_Color: 0.862745 0.470588 0.352941
Segment0_ColorAutoGenerated: 1
Segment0_Extent: 0 511 0 637 0 196
Segment0_ID: Brain_Parenchyma
Segment0_LabelValue: 1
Segment0_Layer: 0
Segment0_Name: brain parenchyma
Segment0_NameAutoGenerated: 1
Segment0_Tags: TerminologyEntry:Segmentation category and type - MONAI Auto3DSeg~SCT^123037004^Anatomical Structure~SCT^836432005^Parenchyma of brain~^^~Anatomic codes - DICOM master list~^^~^^|
Segment10_Color: 0.0941176 0.533333 0.415686
Segment10_ColorAutoGenerated: 1
Segment10_Extent: 0 511 0 637 0 196
Segment10_ID: Lobar
Segment10_LabelValue: 11
Segment10_Layer: 0
Segment10_Name: Lobar
Segment10_N

type: unsigned char
dimension: 3
space: left-posterior-superior
sizes: [512 512 203]
space directions: [[0.45626172 0.         0.        ]
 [0.         0.45626172 0.        ]
 [0.         0.         0.8       ]]
kinds: ['domain', 'domain', 'domain']
encoding: gzip
space origin: [  -86.59886914  -295.3668691  -1267.494     ]
Segment0_Color: 0.501961 0.682353 0.501961
Segment0_ColorAutoGenerated: 1
Segment0_Extent: 144 361 145 364 73 162
Segment0_ID: Segment_1
Segment0_LabelValue: 1
Segment0_Layer: 0
Segment0_Name: Segment_1
Segment0_NameAutoGenerated: 1
Segment0_Tags: Segmentation.Status:inprogress|TerminologyEntry:Segmentation category and type - 3D Slicer General Anatomy list~SCT^85756007^Tissue~SCT^85756007^Tissue~^^~Anatomic codes - DICOM master list~^^~^^|
Segment1_Color: 0.945098 0.839216 0.568627
Segment1_ColorAutoGenerated: 1
Segment1_Extent: 144 361 145 364 73 162
Segment1_ID: Segment_2
Segment1_LabelValue: 2
Segment1_Layer: 0
Segment1_Name: Segment_2
Segment1_NameAutoGenerated

In [ ]:
import nibabel as nib
import numpy as np

# Path to your NIfTI segmentation file
file_path = '/content/drive/MyDrive/Neuro/CODE/seg/1075451.nii.gz'

# Load the NIfTI file
nifti_img = nib.load(file_path)

# Get the data array from the NIfTI file
data_array = nifti_img.get_fdata()

# Find unique labels in the segmentation
unique_labels, counts = np.unique(data_array, return_counts=True)

# Display the labels and their counts
for label, count in zip(unique_labels, counts):
    print(f"Label: {label}, Count: {count}")


In [ ]:
import os
import glob
import nibabel as nib
import numpy as np

# Specify the directory containing NIfTI files
input_dir = '/content/drive/MyDrive/Neuro/CODE/seg'

# Get a list of NIfTI files in the directory
nifti_files = glob.glob(os.path.join(input_dir, '*.nii.gz'))

# Check if there are any files to process
if not nifti_files:
    print("No NIfTI files found in the specified directory.")
    exit()

# Dictionary to hold labels from each file
file_labels = {}

# Process each file
for file_path in nifti_files:
    # Load the NIfTI file
    nifti_img = nib.load(file_path)
    data_array = nifti_img.get_fdata()

    # Find unique labels in the segmentation
    unique_labels = np.unique(data_array)

    # Store the labels for this file
    file_labels[os.path.basename(file_path)] = set(unique_labels)

# Determine the set of common labels (intersection of all sets)
if file_labels:
    common_labels = set.intersection(*file_labels.values())
else:
    print("No labels found across files.")
    exit()

# Identify and report discrepancies
discrepancies = {}
for filename, labels in file_labels.items():
    if labels != common_labels:
        discrepancies[filename] = {
            'missing_labels': list(common_labels - labels),
            'extra_labels': list(labels - common_labels)
        }

# Output the discrepancies
if discrepancies:
    print("Discrepancies found in labels among files:")
    for filename, info in discrepancies.items():
        print(f"{filename}: Missing Labels: {info['missing_labels']}, Extra Labels: {info['extra_labels']}")
else:
    print("No discrepancies found. All files have the same set of labels.")


In [ ]:
print(common_labels)

In [ ]:
# Get a list of .nrrd files in a directory
nrrd_files = glob.glob('/content/drive/MyDrive/Neuro/CODE/seg/*.nrrd')

# Loop through each .nrrd file
for nrrd_file in nrrd_files:
    # Read the .nrrd file
    data, header = nrrd.read(nrrd_file)

    # Create a NIfTI1Image object
    nifti_img = nib.Nifti1Image(data, affine=None)

    # Update the NIfTI header with necessary information
    nifti_img.header.set_data_dtype(data.dtype)
    nifti_img.header.set_zooms([header['space directions'][0, 0],
                            header['space directions'][1, 1],
                            header['space directions'][2, 2]])

    # Generate the output .nii file path by replacing the extension
    nii_file = nrrd_file.replace('.nrrd', '.nii.gz')

    # Save the NIfTI1Image object as .nii file
    nib.save(nifti_img, nii_file)

In [ ]:
import nrrd
import os

# Path to the input directory
input_dir = '/content/drive/MyDrive/Neuro/CODE/seg'

# List all nrrd files in the input directory
nrrd_files = [file for file in os.listdir(input_dir) if file.endswith('.nrrd')]

# Load the first nrrd file (for testing)
file_path = os.path.join(input_dir, nrrd_files[0])
data, header = nrrd.read(file_path)

# Print out the header to explore metadata
print(header)


#other FUNCTIONs FOR CONVERTING A NRRD FILE INTO A .NII.GZ
If you want the uncompressed version (.NII) just change "output_file_name" ending in line 16 with ".nii" (instead of .nii.gz)

In [ ]:
import numpy as np
import nibabel as nib

# Output directory for the NIfTI files
output_dir = '/content/drive/MyDrive/Neuro/CODE/out'

# Labels we are interested in (example: 'brain parenchyma', 'thalamus')
interested_labels = ['lobar']

def save_label_as_nifti(data, header, label_value, output_dir, original_file_name, label_name):
    # Create a binary mask for the current label
    label_mask = data == label_value
    # Check if the mask has any true values
    if np.any(label_mask):
        # Construct the output file name
        output_file_name = f"{original_file_name}_{label_name.replace(' ', '_')}.nii.gz"
        print(f"Saving {output_file_name} with value {label_value}")
        # Convert binary mask to NIfTI image
        nifti_img = nib.Nifti1Image(label_mask.astype(np.int16), affine=np.eye(4))
        # Save the NIfTI image
        nib.save(nifti_img, os.path.join(output_dir, output_file_name))
    else:
        print(f"No data found for label {label_name} with value {label_value}")

for file in nrrd_files:
    print(f"Processing file: {file}")
    file_path = os.path.join(input_dir, file)
    data, header = nrrd.read(file_path)
    found_labels = False
    # Extracting the base name without extension to use in naming the output files
    base_name = os.path.splitext(file)[0]

    for i in range(100):  # Adjust based on the number of segments you expect
        label_name_key = f'Segment{i}_Name'
        label_value_key = f'Segment{i}_LabelValue'
        if label_name_key in header and header[label_name_key].lower() in interested_labels:
            label_name = header[label_name_key]
            label_value = int(header[label_value_key])
            save_label_as_nifti(data, header, label_value, output_dir, base_name, label_name)
            found_labels = True

    if not found_labels:
        print(f"No interested labels found in {file}")

In [ ]:
def save_label_as_nifti(data, header, label_value, output_dir, original_file_name, label_name):
    # Create a binary mask for the current label
    label_mask = data == label_value
    # Check if the mask has any true values
    if np.any(label_mask):
        # Print out some stats about the mask
        print(f"Label {label_name} with value {label_value} has {np.sum(label_mask)} true values.")
        # Construct the output file name with .nii.gz extension for compression
        output_file_name = f"{original_file_name}_{label_name.replace(' ', '_')}.nii.gz"
        print(f"Saving {output_file_name} with value {label_value}")
        # Convert binary mask to NIfTI image
        nifti_img = nib.Nifti1Image(label_mask.astype(np.int16), affine=np.eye(4))
        # Save the NIfTI image in compressed format
        nib.save(nifti_img, os.path.join(output_dir, output_file_name))
    else:
        print(f"No data found for label {label_name} with value {label_value}")

for file in nrrd_files:
    print(f"Processing file: {file}")
    file_path = os.path.join(input_dir, file)
    data, header = nrrd.read(file_path)
    found_labels = False
    # Extracting the base name without extension to use in naming the output files
    base_name = os.path.splitext(file)[0]

    for i in range(100):  # Adjust based on the number of segments you expect
        label_name_key = f'Segment{i}_Name'
        label_value_key = f'Segment{i}_LabelValue'
        if label_name_key in header and header[label_name_key].lower() in interested_labels:
            label_name = header[label_name_key]
            label_value = int(header[label_value_key])
            save_label_as_nifti(data, header, label_value, output_dir, base_name, label_name)
            found_labels = True

    if not found_labels:
        print(f"No interested labels found in {file}")